Dataset downloaded from here: https://github.com/wuhuikai/MSC

To reproduce notebooks, change absolute paths accordingly

In [1]:
zvz = "/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Zerg_vs_Zerg/Zerg/"
pvp = "/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Protoss_vs_Protoss/Protoss/"
tvt = "/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/"

matchups = [zvz, pvp, tvt]

to reproduce for races zerg and protoss, rewrite the following code block

In [7]:
import os
import shutil

allFileNames = os.listdir(tvt)
#TODO: This is bad and ugly, rewrite this
for i in range(0, int(len(allFileNames) * 0.10)):
    if ".npz" in allFileNames[i]:
        shutil.move(tvt + allFileNames[i], tvt + "/test")

for i in range(int(len(allFileNames) * 0.10), int(len(allFileNames) * 0.2)):
    if ".npz" in allFileNames[i]:
        shutil.move(tvt + allFileNames[i], tvt + "/validation")

for i in range(int(len(allFileNames) * 0.2), int(len(allFileNames))):
    if ".npz" in allFileNames[i]:
        shutil.move(tvt + allFileNames[i], tvt + "/train")

In [6]:
# get lengths of each split

allFileNames = os.listdir(tvt + "train/")
print("training set size: " + len(allFileNames))
allFileNames = os.listdir(tvt + "test/")
print("test set size: " + len(allFileNames))
allFileNames = os.listdir(tvt + "validation/")
print("validation set size: " + len(allFileNames))

NotADirectoryError: [Errno 20] Not a directory: '/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Zerg_vs_Zerg/Zerg/train/'

Create H5 training file, used for training LSTM on Google Colab. This is done as loading individual replays from Drive is exceedingly slow, H5 format allows loading replays in batches without keeping the entire file in memory.

In [ ]:
sets = ["train", "test", "validation"]

for name in sets:
    d = (
        "/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/"
        + name
    )
    replays = []

    for r, d, f in os.walk(d):
        for file in f:
            if ".npz" in file:
                path = os.path.join(r, file)
                # files.append(os.path.join(r, file)
                replays.append(sparse.load_npz(path))

    replays = np.asarray(replays)

    # create h5 object
    hf = h5sparse.File(f"./datasets/{name}.h5", "w")

    # store each replay as sparse matrix
    for i, replay in enumerate(replays):
        hf.create_dataset(f"sparse/{i}", data=replay)

    hf.close()